<a href="https://colab.research.google.com/github/Pablo1990/tyssue/blob/main/notebooks/basic_usage_quick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install numpy==1.23.5 scipy matplotlib pandas==1.5.3 jupyter notebook quantities ipywidgets pythreejs ipyvolume vispy tyssue==0.8.0

# Auxiliar functions

In [ ]:
from tyssue import History
from tyssue.draw import sheet_view
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.io import obj
import csv
import matplotlib.pylab as plt
import numpy as np
import pathlib
import random
import numpy as np

import matplotlib.pylab as plt
from tyssue.draw import sheet_view
from tyssue.io import obj


def line_tension_range(cellmap, lower_line_tension, higher_line_tension):
    """
    Randomly assign line tension values between lower_line_tension and higher_line_tension
    :param cellmap:
    :param lower_line_tension:
    :param higher_line_tension:
    :return:
    """
    for edge in range(len(cellmap.edge_df)):
        newValue = random.uniform(lower_line_tension, higher_line_tension)
        cellmap.edge_df.loc[edge, 'line_tension'] = newValue
    return cellmap

def length_elasticity_range(cellmap, lower_line_tension, higher_line_tension):
    for edge in range(len(cellmap.edge_df)):
        newValue = random.uniform(lower_line_tension, higher_line_tension)
        cellmap.edge_df.loc[edge, 'length_elasticity'] = newValue
    return cellmap

def create_frames(
    history,
    output,
    num_frames=None,
    interval=None,
    margin=0,
    **draw_kwds,
):
    """
    Creates a set of png frames of the recorded history.
    :param history:  a :class:`tyssue.History` object
    :param output:  path to the output directory
    :param num_frames:  int, the number of frames in the gif
    :param interval:    tuples, define begin and end frame of the gif
    :param draw_func:   a drawing function. This function must take a `sheet` object as first argument and return a
    `fig, ax` pair. Defaults to quick_edge_draw(aka sheet_view with quick mode)
    :param margin:  int, the graph margins in percents, default 5. If margin is -1, let the draw function decide
    :param draw_kwds:   are passed to the drawing function
    :return:
    """

    graph_dir = pathlib.Path(output)
    graph_dir.mkdir(parents=True, exist_ok=True)

    x, y = coords = draw_kwds.get("coords", history.sheet.coords[:2])
    sheet0 = history.retrieve(0)
    bounds = sheet0.vert_df[coords].describe().loc[["min", "max"]]
    delta = (bounds.loc["max"] - bounds.loc["min"]).max()
    margin = delta * margin / 100
    xlim = bounds.loc["min", x] - margin, bounds.loc["max", x] + margin
    ylim = bounds.loc["min", y] - margin, bounds.loc["max", y] + margin

    if interval is None:
        start, stop = None, None
    else:
        start, stop = interval[0], interval[1]

    # Replace the loop that uses 'browse' with this manual approach
    for i, t in enumerate(np.linspace(start if start else 0, stop if stop else history.time,
                                      num_frames if num_frames else len(history.time_stamps))):
        try:
            # Manually retrieve each sheet for the corresponding time step
            sheet = history.retrieve(int(t))  # retrieve from history at time 't'

            # Now apply the sheet view function as before
            fig, ax = sheet_view(sheet, **draw_kwds)
            fig.set_size_inches(20, 20)

            # if isinstance(ax, plt.Axes) and margin >= 0:
            #     ax.set(xlim=xlim, ylim=ylim)

            # labels = np.array(sheet.face_df.index.array, dtype=np.uint32);
            # x = sheet.face_df.x;
            # y = sheet.face_df.y;
            # for index, label in enumerate(labels):
            #     ax.text(x[index], y[index], label, fontsize=5, ha='center')

            # Combine the arrays into a list of rows
            rows = zip(sheet.face_df.x, bounds.loc["max", y] - sheet.face_df.y, np.array(sheet.face_df.index.array, dtype=np.uint32))

            # Define the file path and name
            file_name = graph_dir / f"movie_{i:04d}.csv"

            # # Open the CSV file in write mode and write the rows
            # with open(file_name, 'w', newline='') as file:
            #     writer = csv.writer(file)
            #     writer.writerow(['Column1', 'Column2', 'Column3'])  # Write header
            #     writer.writerows(rows)  # Write data rows

            plt.axis('off')
            fig.savefig(graph_dir / f"movie_{i:04d}.png") #, bbox_inches='tight', pad_inches=0
        except Exception as e:
            print(f"Dropped frame {i}")
            print(e)

        plt.close()


def exportToMesh(history, dir):
    """
    Exporting each timepoint to mesh
    :param history:
    :param dir:
    :return:
    """

    # Replace the loop that uses 'browse' with this manual approach
    for i, t in enumerate(np.linspace(0, history.time, len(history.time_stamps))):
        try:
            # Manually retrieve each sheet for the corresponding time step
            sheet = history.retrieve(int(t))  # retrieve from history at time 't'

            # Save the split cells data using the provided function
            obj.save_splitted_cells(dir + f'/junctions_{int(t)}.obj', sheet, epsilon=0.001)

        except Exception as e:
            print(f"Error at time {t}: {e}")

# Brownian motion

In [ ]:
import numpy as np
import pandas as pd
from tyssue.dynamics import effectors


# Adding some gigling
class BrownianMotion(effectors.AbstractEffector):

    label = 'Brownian Motion'
    element = 'vert'
    specs = {"settings": {"temperature": 1e-3}}

    def energy(eptm):
        T = eptm.settings['temperature']
        return np.ones(eptm.Nv) * T / eptm.Nv

    def gradient(eptm):
        T = eptm.settings['temperature']
        scale = T/eptm.edge_df.length.mean()

        grad = pd.DataFrame(
            data=np.random.normal(0, scale=scale, size=(eptm.Nv, eptm.dim)),
            index=eptm.vert_df.index,
            columns=['g'+c for c in eptm.coords]
        )
        return grad, None

# Vertex model

In [ ]:
import copy

from tyssue import PlanarGeometry, Sheet, History
from tyssue.behaviors import EventManager
from tyssue.draw import sheet_view
from tyssue.dynamics import effectors, model_factory
from tyssue.solvers.viscous import EulerSolver

def initialize():
    """
    Initialize the model
    :return:
    """
    ## Defining energy contributions
    # https://tyssue.readthedocs.io/en/latest/_modules/tyssue/dynamics/effectors.html
    energyContributions_model = model_factory([
        BrownianMotion,
        effectors.FaceAreaElasticity,
        effectors.LineTension,
        # effectors.LengthElasticity,
        effectors.PerimeterElasticity,
        # effectors.CellAreaElasticity,
        # effectors.FaceContractility,
        # effectors.BarrierElasticity
        # effectors.LineViscosity
        # effectors.BorderElasticity
    ])

    ## Size of the patch
    numCellRows = 40
    noiseCellShape = 0.2

    # noise = 0 -> hexagonal pattern
    # noise = 1 -> random voronoi
    cellMap = Sheet.planar_sheet_2d('tissue',
                                    nx=numCellRows,  # approximate number of cells on the x axis
                                    ny=numCellRows,  # approximate number of cells along the y axis
                                    distx=1,  # distance between 2 cells along x
                                    disty=1,  # distance between 2 cells along y
                                    noise=noiseCellShape)

    try:
        cellMap.remove(cellMap.cut_out([[1, numCellRows], [1, numCellRows]]), trim_borders=True)
    except:
        cellMap.sanitize(trim_borders=True, order_edges=True)
    cellMap.reset_index()
    cellMap.reset_topo()

    # Definition of the geometry of the sheet
    # PlanarGeometry: Geometry methods for 2D planar cell arangements
    # SheetGeometry: Geometry definitions for 2D sheets in 3D
    # BulkGeometry: Geometry functions for 3D cell arangements
    geom = PlanarGeometry

    # Update geometry with the patch
    geom.update_all(cellMap)

    # Visualize the sheet
    fig, ax = sheet_view(cellMap, mode="quick")

    ## Connect cells with energy contributions
    cellMap.update_specs(energyContributions_model.specs)

    return [cellMap, geom, energyContributions_model]


def on_topo_change(sheet):
    """

    :param sheet:
    :return:
    """
    print('Topology changed!')


def solveEuler(cellMap, geom, energyContributions_model, endTime):
    """

    :param cellMap:
    :param geom:
    :param energyContributions_model:
    :param endTime:
    :return:
    """
    ## Init history object
    # The History object records all the time steps
    history_cellMap = History(cellMap)

    ## Manager Initialization
    manager = EventManager("face", )
    # manager.append(basic_events.auto_reconnect)

    ## Init solver
    solver1 = EulerSolver(cellMap,
                          geom,
                          energyContributions_model,
                          manager=manager,
                          bounds=(
                              -cellMap.edge_df.length.median() / 10,
                              cellMap.edge_df.length.median() / 10
                          ),
                          history=history_cellMap,
                          auto_reconnect=True)

    manager.update()

    ## Run the solver
    res1 = solver1.solve(tf=endTime, dt=1, on_topo_change=on_topo_change,
                         topo_change_args=(solver1.eptm,))

    ## Deep copy to return it and being able to modify maintaining the previous one
    cellMap_new = copy.deepcopy(cellMap)
    history_new = copy.deepcopy(history_cellMap)

    return [cellMap_new, geom, energyContributions_model, history_new]


def solveStepByStep(cellMap, geom, energyContributions_model, endTime):
    """

    :param cellMap:
    :param geom:
    :param energyContributions_model:
    :param endTime:
    :return:
    """
    ## Init history object
    # The History object records all the time steps
    history_cellMap = History(cellMap)

    ## Manager Initialization
    manager = EventManager("manager", )

    ## Find the minima in different timeSteps:
    # https://tyssue.readthedocs.io/en/latest/notebooks/07-EventManager.html
    t = 0

    history_cellMap = history_cellMap(cellMap)

    while manager.current and t < endTime:
        # Execute the event in the current list
        manager.execute(cellMap)
        t += 1
        cellMap.reset_index(order=True)
        # Find energy min
        res = solver.resolve_t1s(cellMap, geom, energyContributions_model)
        history_cellMap.record()
        # Switch event list from the next list to the current list
        manager.update()

    ## Deep copy to return it and being able to modify maintaining the previous one
    cellMap_new = copy.deepcopy(cellMap)
    history_new = copy.deepcopy(history_cellMap)

    return [cellMap_new, geom, energyContributions_model, history_new]


collision solver could not be imported You may need to install CGAL and re-install tyssue


# Mechanical parameters

In [ ]:
def update(cellmap):
    """

    :param cellmap:
    :return:
    """
    # Per sheet:
    cellmap.settings["temperature"] = 10
    # Stochastically detaches vertices from rosettes.
    # Uses two probabilities `p_4` and `p_5p`
    cellmap.settings["p_4"] = 10
    cellmap.settings["p_5p"] = 0.1
    cellmap.settings["threshold_length"] = 2e-2

    # Per cell:

    # Per face:
    cellmap.face_df["area_elasticity"] = 50
    cellmap.face_df["prefered_area"] = cellmap.face_df["area"].mean() * 1.1

    cellmap.face_df["perimeter"] = 1
    cellmap.face_df["perimeter_elasticity"] = 10
    cellmap.face_df["prefered_perimeter"] = 3.81

    # Per edge:
    rangeLineTension = 0

    if rangeLineTension:
        lower_line_tension = 0.1
        higher_line_tension = 1
        cellmap = line_tension_range(cellmap, lower_line_tension, higher_line_tension)
    else:
        cellmap.edge_df["line_tension"] = 0.1

    # Per vertex:
    cellmap.vert_df["viscosity"] = 1

    return cellmap


# Main

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from tyssue.draw import sheet_view

In [ ]:
# Initialize Model
[cellmap_init, geom, energyContributions_model] = initialize()

# Update mechanical parameters
cellmap_init = update(cellmap_init)

# Initial stage
energyContributions_model.compute_energy(cellmap_init)

In [ ]:
# RUN
[cellmap_H, geom, model_H, history_H] = solveEuler(cellmap_init, geom, energyContributions_model, endTime = 100)

In [ ]:
cellmap_H.edge_df["color_edges"] = cellmap_H.edge_df['length']/(max(cellmap_H.edge_df['length']))
cellmap_H.face_df["color_edges"] = cellmap_H.face_df['area']/(max(cellmap_H.face_df['area']))

## https://tyssue.readthedocs.io/en/latest/notebooks/02-Visualization.html
specs = {
    'face': {
        'visible': True,
        'color': cellmap_H.face_df["color_edges"],
    },
    'edge': {
        'visible': True,
        'color': cellmap_H.edge_df["color_edges"],
        'colormap':'Greys',
        'width': 1,
    },
    'vert': {
        'visible': True,
        's': 5,
    }
}
fig, ax = sheet_view(cellmap_H, **specs)
energyContributions_model.compute_energy(cellmap_init)

In [ ]:
# Create a movie
!rm -rf movie
!mkdir -p movie
create_frames(history_H, 'movie/', edge={'color':'black'})